In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.rdmolops import FindPotentialStereo
import dask.dataframe as dd
from pandas_profiling import ProfileReport
# Alter here only

# Cria a variável que contem a lista de arqquivos que voce quer olhar, se for só um deixa como string
path_to_files = './results'
# To get from a directory                          
# troca os.listdir por um método semelhante que percorra o drive
files = [f"{path_to_files}\\{file}" for file in os.listdir(path_to_files)]
print('\n'.join(files))
print('Arquivos para processar', len(files))
name = [f"{file.strip('.xlsx')}" for file in os.listdir(path_to_files)]
name = [f"{n.strip('.csv')}" for n in name]

your_smiles = "SMILES"
your_values = "Loael (mmol)"
your_study_type = "Loael"
result_name="Sparse_Hepatotox"

C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongoovárioTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongotestículoTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongoúteroTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodisfunçãoeretilTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodisfunçãosexualTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodismenorreiaTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicoejaculaçãoTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicoendométrioTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicogalactorreiaTxlsx.xlsx_default_1.csv
C:\Users\Esousa\Downloads\Data-Treatment-master-tes

Convert to csv 

In [4]:
def convertToCsv():
    for file in files:
        if file.find(".xlsx")!=-1:
            for sheet in pd.ExcelFile(file).sheet_names:
                df = pd.read_excel(file,sheet_name=sheet)
                df.to_csv(f"{file}_{sheet}.csv",index=True)
                print (df)

In [5]:
def read_through_files(files):
        df=[]
        if type(files) is str:
            print('1')
            # name=result_name
            # name=name.replace(".xlsx","")
            # name=name.replace(".csv","")
            if files.find(".csv")!=-1:
                print('1.1')
                df.append(pd.read_csv(files,delimiter=","))
                #organize(df,name)
            else:
                print('1.2')
                sheets = pd.ExcelFile(files).sheet_names
                if len(sheets)>1:
                    print('1.2.1')
                    for sheet in sheets:
                        if sheet != ".":
                            print('1.2.1.1')
                            #name=sheet
                            df.append(pd.read_excel(files,sheet_name=sheet, engine='openpyxl'))
                            #organize(df,name)
                else:
                    print('1.2.2')
                    for sheet in sheets:
                        if sheet != ".":
                            #name=sheet
                            df.append(pd.read_excel(files,sheet_name=sheet, engine='openpyxl'))
        else:
            print('2')
            for file in files:
                if file.find(".csv")!=-1:
                    print('2.1', file)
                    df.append(pd.read_csv(file,delimiter=","))
                else:   
                    print('2.2')
                    sheets = pd.ExcelFile(file).sheet_names
                    if len(sheets)>1:
                        print('2.2.1')
                        for sheet in sheets:
                            if sheet != ".":
                                print('2.2.1.1')
                                #name=sheet
                                df.append(pd.read_excel(file,sheet_name=sheet, engine='openpyxl'))
                                #organize(df,name)
                    else:
                        print('2.2.1')
                        for sheet in sheets:
                            if sheet != ".":
                                print('2.2.1.1')
                                #name=sheet
                                df.append(pd.read_excel(file,sheet_name=sheet, engine='openpyxl'))
        print('leu todos df')
        return df
df = read_through_files(files=files)

2
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongoovárioTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongotestículoTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\camundongoúteroTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodisfunçãoeretilTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodisfunçãosexualTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicodismenorreiaTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicoejaculaçãoTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicoendométrioTxlsx.xlsx_default_1.csv
2.1 C:\Users\Esousa\Downloads\Data-Treatment-master-teste\results\clinicogalactorreiaTxlsx.xlsx_default_1.csv
2.1 C:\Users\

In [6]:
print([len(d) for d in df])
print(sum([len(d) for d in df]))

[20, 11, 75, 204, 200, 277, 459, 91, 76, 306, 86, 45, 806, 222, 309, 139, 117, 55, 121, 193, 337, 98, 119, 116, 66, 184, 49, 78, 82, 234, 142, 27, 33, 320, 78, 69, 214]
6058


In [7]:
def organize(all_df):
    #smiles_list=[]
    df_smiles=[]
    all_smiles = []
    for i,df in enumerate(all_df):
        endpoint = []
        df_loc = df.loc[:,[your_smiles, your_values]]
        smiles = df_loc[your_smiles]
        measures = df_loc[your_values]
        canon_smiles = []
        inchik = []
        for j,ds in enumerate(smiles):
            try:
                try:
                    cs = Chem.CanonSmiles(ds)
                    mol = Chem.MolFromSmiles(cs)
                    local_inchi = Chem.rdinchi.MolToInchiKey(mol)
                    inchik.append(local_inchi)
                    endpoint.append({
                        'inchik': local_inchi,
                        'measure': measures[j]
                    })
                    canon_smiles.append(cs)
                    all_smiles.append(local_inchi)
                except:
                    df.drop(index=i,drop=True,inplace=True)
                    df.reset_index(drop=True, inplace=True)
            except:
                print('falha em', i, ds)
            

        df_smiles.append(endpoint)

    return [all_smiles, df_smiles]

In [10]:
[all_smiles, df_smiles] = organize(df)
to_data = {
    'inchik' : all_smiles
}
ends = []
for item in range(0, len(df_smiles)):
    key = item
    values = list(map((lambda x: x['inchik']), df_smiles[key]))

    end_name = 'loael_' + str(key)
    ends.append(end_name)
    to_data[end_name] = []
    for v in all_smiles:
        val = next(filter(lambda sm: sm['inchik'] == v, df_smiles[key]), {'measure':0})['measure']
        to_data[end_name].append(val)
        
new_data = pd.DataFrame(to_data, index=all_smiles)
print(new_data)

falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 18 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 22 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 22 CC(C)[N]1(C)C2CCC1CC(C2)OC(=O)C(CO)C1=CC=CC=C1
falha em 22 C[N]1(CC2CC2)CCC23C4OC5=C(O)C=CC(CC1C2(O)CCC4=O)=C35
falha em 23 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 23 CN(C)C(=O)OC1=C[N](C)=CC=C1
falha em 25 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 25 C[N]1(C)CCC(C1)OC(=O)C(O)(C1CCCC1)C1=CC=CC=C1
falha em 25 CC(C)[N]1(C)C2CCC1CC(C2)OC(=O)C(CO)C1=CC=CC=C1
falha em 26 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 27 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha em 29 CCCCCCCCCCCCCCCCOP(O)(=O)OCC[N](C)(C)C
falha

In [11]:
new_data.to_csv(f"./results/{result_name}.csv",index=False)

In [12]:
correlation_df=pd.read_csv(f"./results/{result_name}.csv")
corr=correlation_df.corr()
sns.set_style('darkgrid')
sns.set(font_scale=0.9, rc={'figure.figsize':(12,10)})
sns.heatmap(corr, cmap='RdYlBu_r', vmin=-1, vmax=1, annot=True)
plt.title('Correlation Heatmap', fontsize=2)
plt.savefig('Correlation.png', bbox_inches='tight', transparent=True, format='png', dpi=400)